In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv

from ML.DTGenerated import DTGenerated
import pickle

In [9]:
from generateTestingData.useGeneratedData import makeDataUsable, splitTrainTest, NewPredicter

In [10]:
MODEL_PREFIX = "gen"
PI = 4 # past importance
RBC = 5 # rows before col

In [7]:
df = makeDataUsable(read_csv("../data/generatedData.csv"), PI, RBC)
print(df.shape)
df.head()

(3950, 30)


,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,50.0,50.0,50.0,50.000,16.6264,100.0,100.0,100.0000,0.004,...,0.000,0.000,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,0.0
1,4.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.661,...,-0.001,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,-0.001,0.0
2,5.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,4.117,...,-0.000,-0.001,-0.001,0.000,0.000,0.001,0.000,-0.001,0.000,0.0
3,6.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.865,...,0.000,-0.001,0.000,0.000,0.000,0.000,-0.001,0.000,0.000,0.0
4,7.0,50.0,50.0,50.0,5.322,100.0000,100.0,100.0,83.3296,3.704,...,0.004,0.000,0.000,0.000,-0.001,-0.001,0.000,0.000,0.005,0.0


In [11]:
df["COL"].value_counts()

0.0    3458
1.0     492
Name: COL, dtype: int64

In [12]:

# sampleSize = int(len(data[data['COL'] == 1]) * undersampleRatio)
# print("Undersampling")
# data = concat([data[data["COL"] == 0].sample(sampleSize, random_state=1), data[data["COL"] == True]])

TODO: need to either get more crash data or keep undersample below

In [13]:
df = pd.concat([df[df["COL"] == False].sample(500, random_state=1), df[df["COL"] == True]])

In [14]:
data = df.copy().drop(columns=df.columns[0], axis=1)
trainX, trainY, testX, testY = splitTrainTest(data)

Splitting at 793, total rows: 992


In [15]:
trainX.head(3)

,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,...,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4
1302,50.000,50.000,50.000,50.000,6.1382,6.9333,8.1275,9.6133,0.099,0.035,...,0.0,0.003,0.0,-0.207,0.088,0.013,-0.0,0.036,-0.0,-0.003
2365,50.000,50.000,50.000,50.000,1.7815,3.1742,4.5489,5.8832,19.134,1.000,...,0.0,-0.000,-0.0,-0.000,-0.000,-0.000,-0.0,-0.000,0.0,0.000
2641,8.398,8.551,8.794,9.336,20.0793,19.0091,18.0272,17.1499,0.000,0.024,...,-0.0,-0.002,0.0,0.000,-0.023,-0.000,-0.0,-0.000,0.0,0.002


## Model

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

### MLP Classifier

In [17]:
mlp_clf = NewPredicter()
mlp_trainX = mlp_clf.preProcess(trainX.copy())
mlp_testX = mlp_clf.preProcess(testX.copy())

mlp_trainY, mlp_testY = trainY.to_numpy(), testY.to_numpy()

mlp_clf.fit(mlp_trainX, mlp_trainY)

Scaler is fitted


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
mlp_pred = mlp_clf.predict(mlp_testX)
mlp_score = mlp_clf.getScore(mlp_pred, mlp_testY)

Total: 199, Collisions: 90
Accuracy: 0.7185929648241206, Precision: 0.6666666666666666, Recall: 0.7555555555555555, F1: 0.7083333333333334
Confusion matrix:
[[75 34]
 [22 68]]


In [19]:
model_score = f"{mlp_score[0][0]}-{mlp_score[0][1]}-{mlp_score[1][0]}-{mlp_score[1][1]}"
mlp_clf.saveModel(f"MLPClassifier_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!


### Random Forest Classifier

In [20]:
rf_clf = NewPredicter(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1))
rf_trainX = rf_clf.preProcess(trainX.copy())
rf_testX = rf_clf.preProcess(testX.copy())

rf_trainY, rf_testY = trainY.to_numpy(), testY.to_numpy()

rf_clf.fit(rf_trainX, rf_trainY)

Scaler is fitted


In [21]:
rf_pred = rf_clf.predict(rf_testX)
rf_score = rf_clf.getScore(rf_pred, rf_testY)

Total: 199, Collisions: 83
Accuracy: 0.7537688442211056, Precision: 0.6666666666666666, Recall: 0.8192771084337349, F1: 0.7351351351351351
Confusion matrix:
[[82 34]
 [15 68]]


In [22]:
model_score = f"{rf_score[0][0]}-{rf_score[0][1]}-{rf_score[1][0]}-{rf_score[1][1]}"
rf_clf.saveModel(f"RandomForestClassifier_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!


### SVM Classifier

In [23]:
svm_clf = NewPredicter(SVC(random_state=1))
svm_trainX = svm_clf.preProcess(trainX.copy())
svm_testX = svm_clf.preProcess(testX.copy())

svm_trainY, svm_testY = trainY.to_numpy(), testY.to_numpy()

svm_clf.fit(svm_trainX, svm_trainY)

Scaler is fitted


In [24]:
svm_pred = svm_clf.predict(svm_testX)
svm_score = svm_clf.getScore(svm_pred, svm_testY)

Total: 199, Collisions: 79
Accuracy: 0.7336683417085427, Precision: 0.6274509803921569, Recall: 0.810126582278481, F1: 0.7071823204419889
Confusion matrix:
[[82 38]
 [15 64]]


In [25]:
model_score = f"{svm_score[0][0]}-{svm_score[0][1]}-{svm_score[1][0]}-{svm_score[1][1]}"
svm_clf.saveModel(f"SVC_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!


### XGBoost Classifier

In [26]:
xgb_clf = NewPredicter(XGBClassifier(n_estimators=100, random_state=1))
xgb_trainX = xgb_clf.preProcess(trainX.copy())
xgb_testX = xgb_clf.preProcess(testX.copy())

xgb_trainY, xgb_testY = trainY.to_numpy(), testY.to_numpy()

Scaler is fitted


In [27]:
xgb_clf.fit(xgb_trainX, xgb_trainY)

In [28]:
len(xgb_testX[0])
xgb_pred = xgb_clf.predict(xgb_testX)
xgb_score = xgb_clf.getScore(xgb_testY, xgb_pred)

Total: 199, Collisions: 102
Accuracy: 0.7889447236180904, Precision: 0.8333333333333334, Recall: 0.7352941176470589, F1: 0.78125
Confusion matrix:
[[82 15]
 [27 75]]


TODO - need more data, what is radar?

In [29]:
# model_score = f"{score[0][0]}-{score[0][1]}-{score[1][0]}-{score[1][1]}"
model_score = f"{xgb_score[0][0]}-{xgb_score[0][1]}-{xgb_score[1][0]}-{xgb_score[1][1]}"
xgb_clf.saveModel(f"XGBClassifier_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!
